In [ ]:
import json
with open('properties.json', 'r') as f:
    properties = json.load(f)
properties

In [ ]:
brokers = '{}-cp-kafka:9092'.format(properties['pnda-helm-release'])
topic = properties['topic']

In [ ]:
import random
import time
from kafka import KafkaProducer

def openmetrics_producer(stop_event,brokers, topic, metric, rate):
    producer = KafkaProducer(bootstrap_servers=brokers.split())
    print("Starting Streaming {} to Kafka: rate {} seconds.".format(metric,rate))
    while not stop_event.wait(rate):
        openmetric = '{metric} {value:.2f}'.format(metric=metric, value=random.uniform(0,50))
        future = producer.send(
            topic,
            key=metric.encode('utf-8'),
            value= openmetric.encode('utf-8'))
        # Block for 'synchronous sends'
        record_metadata = future.get(timeout=1)

In [ ]:
import threading

producer = KafkaProducer(bootstrap_servers=brokers.split())

future = producer.send(
            topic,
            key='temperature{"room"="bathroom"}'.encode('utf-8'),
            value= 'temperature{"room"="bathroom"} 40.0'.encode('utf-8'))
# Block for 'synchronous sends'
record_metadata = future.get(timeout=1)

In [ ]:
import threading

kill_event = threading.Event()
job = threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'temperature{"room"="bedroom"}',0.6])
job.start()

try:
    while True:
        pass
except KeyboardInterrupt:
    print("stopping openmetrics generators")
    kill_event.set()
    job.join()